In [3]:
# Install dependencies
!pip install -q geopandas folium pycountry Pillow selenium


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
# Download data
!mkdir data/ && wget -O ./data/SA_0000001439.csv "https://apps.who.int/gho/athena/api/GHO/SA_0000001439?format=csv"

In [1]:
import folium
import pandas as pd
import pycountry
import io
from PIL import Image

# load data
cancer_data = pd.read_csv("../data/SA_0000001439.csv")
political_countries_url = (
    "http://geojson.xyz/naturalearth-3.3.0/ne_50m_admin_0_countries.geojson"
)

# convert country code to full name
tmp = []
for x in cancer_data["COUNTRY"]:
    try:
        tmp.append(pycountry.countries.get(alpha_3=x).name)
    except AttributeError:
        tmp.append(x)
cancer_data["COUNTRY"] = tmp

# create world map and layout
m = folium.Map(location=(30, 10), zoom_start=2, tiles="cartodb positron")
folium.Choropleth(
    geo_data=political_countries_url,
    data=cancer_data,
    columns=["COUNTRY", "Numeric"],
    key_on="feature.properties.name",
    fill_color="Reds",
    nan_fill_color="Grey",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Breast cancer mortality rate"
).add_to(m)

# to export snapshot of map as PNG
img_data = m._to_png(5)
img = Image.open(io.BytesIO(img_data))
img.save('mortality_rate_breast_cancer.png')

# to export as interactive viewer in HTML
m.save("mortality_rate_breast_cancer.html")

# render map in colab directly
m


KeyError: "None of ['Population'] are in the columns"